# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cityWeatherDF = pd.read_csv("../output_data/city_weather.csv")
cityWeatherDF.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Dunedin,-45.8742,170.5036,59.29,82,100,2.66,NZ,1640940242
1,Qaanaaq,77.4840,-69.3632,-10.35,69,97,7.40,GL,1640940243
2,Kapaa,22.0752,-159.3190,73.72,86,90,1.99,US,1640940243
3,Bredasdorp,-34.5322,20.0403,77.29,49,0,14.97,ZA,1640940244
4,Rikitea,-23.1203,-134.9692,77.36,75,94,11.88,PF,1640940245


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Access maps with unique API key
gmaps.configure(api_key = g_key)

In [4]:
cities = cityWeatherDF['City']
locations = cityWeatherDF[['Lat', 'Lng']]
weights = cityWeatherDF['Humidity']

In [14]:
# Customize the size of the figure
figure_layout = {
    'width': '1500px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, center = (40.71427, -74.00597), zoom_level = 4)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
# Ideal weather conditions are considered to be Max Temperature between 65F and 80F with 
#                Wind speed less than 10 mph and zero cloudiness

idealCityWeatherDF = cityWeatherDF.loc[((cityWeatherDF['Max Temp'] < 80) & (cityWeatherDF['Max Temp'] > 65))] \
                                .loc[(cityWeatherDF['Wind Speed'] < 10)] \
                                .loc[(cityWeatherDF['Cloudiness'] == 0)] \
                                .dropna()
idealCityWeatherDF

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
34,Hermanus,-34.4187,19.2345,77.81,53,0,2.77,ZA,1640940261
38,Puerto del Rosario,28.5004,-13.8627,67.68,67,0,4.61,ES,1640940263
40,Los Llanos de Aridane,28.6585,-17.9182,73.15,35,0,8.99,ES,1640940264
78,Saldanha,-33.0117,17.9442,73.45,59,0,8.90,ZA,1640940286
110,Mungeli,22.0667,81.6833,77.04,41,0,2.98,IN,1640940307
156,Kayes,14.0000,-11.0000,77.85,12,0,7.36,ML,1640940332
167,Nouadhibou,20.9310,-17.0347,66.18,52,0,9.22,MR,1640940340
177,Cape Town,-33.9258,18.4232,76.12,51,0,9.22,ZA,1640940146
186,Coahuayana Viejo,18.7333,-103.6833,67.59,92,0,3.20,MX,1640940350
216,San Juan,-31.5375,-68.5364,73.26,30,0,5.73,AR,1640940369


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
hotel_df = idealCityWeatherDF.reset_index(drop=True)
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Hermanus,-34.4187,19.2345,77.81,53,0,2.77,ZA,1640940261,
1,Puerto del Rosario,28.5004,-13.8627,67.68,67,0,4.61,ES,1640940263,
2,Los Llanos de Aridane,28.6585,-17.9182,73.15,35,0,8.99,ES,1640940264,
3,Saldanha,-33.0117,17.9442,73.45,59,0,8.90,ZA,1640940286,
4,Mungeli,22.0667,81.6833,77.04,41,0,2.98,IN,1640940307,
5,Kayes,14.0000,-11.0000,77.85,12,0,7.36,ML,1640940332,
6,Nouadhibou,20.9310,-17.0347,66.18,52,0,9.22,MR,1640940340,
7,Cape Town,-33.9258,18.4232,76.12,51,0,9.22,ZA,1640940146,
8,Coahuayana Viejo,18.7333,-103.6833,67.59,92,0,3.20,MX,1640940350,
9,San Juan,-31.5375,-68.5364,73.26,30,0,5.73,AR,1640940369,


In [17]:
# base_url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# geocoordinates
target_radius = 5000
target_type = 'lodging'
target_search = 'hotel'

# Loop through each City to identify the first hotel in the vicinity
for row in hotel_df.index:
    
    try:
        target_coordinates = f"{hotel_df.loc[row, 'Lat']}, {hotel_df.loc[row, 'Lng']}"

        # set up a parameters dictionary
        params = {
            "key": g_key,
            "location": target_coordinates,
            "radius": target_radius,
            "type": target_type,
            "keyword": target_search
        }

        # run a request using our params dictionary
        places_data = requests.get(base_url, params=params).json()

        hotel_name = places_data['results'][0]['name']
        print(f"Identified Hotel `{hotel_name}` in the City `{hotel_df.loc[row, 'City']}`")
    except:
        hotel_name = "COULD NOT FIND A MATCH!"
        print(f"SKIPPING.. Could not find hotel in the City `{hotel_df.loc[row, 'City']}`")
    
    hotel_df.loc[row, "Hotel Name"] = hotel_name
    hotel_name = ""

print("Hotel Data loaded successfully!")

Identified Hotel `Misty Waves Boutique Hotel` in the City `Hermanus`
Identified Hotel `Hotel El Mirador` in the City `Puerto del Rosario`
Identified Hotel `Hotel Hacienda de Abajo` in the City `Los Llanos de Aridane`
Identified Hotel `Saldanha Bay Hotel` in the City `Saldanha`
Identified Hotel `Sharma Guest House` in the City `Mungeli`
SKIPPING.. Could not find hotel in the City `Kayes`
Identified Hotel `HOtel Tasiast` in the City `Nouadhibou`
Identified Hotel `Radisson Blu Hotel Waterfront, Cape Town` in the City `Cape Town`
Identified Hotel `Hotel Los Arcos` in the City `Coahuayana Viejo`
Identified Hotel `Hotel Alkazar` in the City `San Juan`
Identified Hotel `Hotel Riu Palace Boavista` in the City `Sal Rei`
Identified Hotel `Mantra Nelson Bay` in the City `Nelson Bay`
Hotel Data loaded successfully!


In [19]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Hermanus,-34.4187,19.2345,77.81,53,0,2.77,ZA,1640940261,Misty Waves Boutique Hotel
1,Puerto del Rosario,28.5004,-13.8627,67.68,67,0,4.61,ES,1640940263,Hotel El Mirador
2,Los Llanos de Aridane,28.6585,-17.9182,73.15,35,0,8.99,ES,1640940264,Hotel Hacienda de Abajo
3,Saldanha,-33.0117,17.9442,73.45,59,0,8.90,ZA,1640940286,Saldanha Bay Hotel
4,Mungeli,22.0667,81.6833,77.04,41,0,2.98,IN,1640940307,Sharma Guest House
5,Kayes,14.0000,-11.0000,77.85,12,0,7.36,ML,1640940332,COULD NOT FIND A MATCH!
6,Nouadhibou,20.9310,-17.0347,66.18,52,0,9.22,MR,1640940340,HOtel Tasiast
7,Cape Town,-33.9258,18.4232,76.12,51,0,9.22,ZA,1640940146,"Radisson Blu Hotel Waterfront, Cape Town"
8,Coahuayana Viejo,18.7333,-103.6833,67.59,92,0,3.20,MX,1640940350,Hotel Los Arcos
9,San Juan,-31.5375,-68.5364,73.26,30,0,5.73,AR,1640940369,Hotel Alkazar


In [20]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [21]:
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', margin='0 auto 0 auto', padding='1px', wi…